In [49]:
import pandas as pd
# !pip install xlrd
file = 'original_all.xls'
dfs = pd.read_html(file, skiprows=5)
df = dfs[0]
# Set the DataFrame columns to the values of the second row (which is at index 1)
df.columns = df.iloc[0]

# Drop the first two rows from the DataFrame as they are no longer needed
df = df.drop(df.index[0:1])

# Reset the index of the DataFrame
df = df.reset_index(drop=True)

# FIX DATE
df['Fecha Documento'] = df['Fecha Documento'].str.replace('=', '')  # Remove "="
df['Fecha Documento'] = df['Fecha Documento'].str.replace('"', '')  # Remove extra quotes

# Step 2: Convert the cleaned strings to datetime objects // Cant be done here needs to be in the importation
# df['Fecha Documento'] = pd.to_datetime(df['Fecha Documento'], format='%d/%m/%Y')

# Fix the int of SKU in the main df
df['SKU'] = df['SKU'].str.replace('=', '')  # Remove "="
df['SKU'] = df['SKU'].str.replace('"', '')  # Remove extra quotes
df['SKU'] = df['SKU'].astype(str)

#Create new sucursal
df = df.dropna(subset=['Sucursal'])
df.loc[(df['Cliente'] == 'UBER PORTIER CHILE SPA'), 'Sucursal'] = 'Uber'
df.loc[(df['Vendedor'] == 'Loadingplay Connect'), 'Sucursal'] = 'Shopify'


# Add brands!
#df shape
df_brands = pd.read_csv('brands.csv')
# merge the two df
df2_with_brands = pd.merge(df, df_brands[['SKU', 'marca']], on='SKU', how='left')
df2_with_brands.loc[df2_with_brands['Producto / Servicio'].str.contains('MELEGATTI', case=False, na=False), 'marca'] = 'Melegatti'

# List of specific 'Producto / Servicio' values to match
productos = [
    'PANDORO CLASSIC ELEGANCE LATA 1 kg',
    'PANETTONE PISTACCHIO LATA 750 g',
    'PANETTONE CLASSICO GOLD LATTA 1 Kg',
    'BISCOTTI AMARETTI MIX LATA RETRO 175 g',
]

# Update 'marca' to 'Lazzaroni' where 'Producto / Servicio' matches any in the list
df2_with_brands.loc[df2_with_brands['Producto / Servicio'].isin(productos), 'marca'] = 'LAZZARONI'

productos = [
    'CREMA CAFFE FREDDA 550 g',
]
# Update 'marca' to 'Lazzaroni' where 'Producto / Servicio' matches any in the list
df2_with_brands.loc[df2_with_brands['Producto / Servicio'].isin(productos), 'marca'] = 'Borbone'

# fill the NaN with 'Other'
df2_with_brands['marca'] = df2_with_brands['marca'].fillna('Other')
#  Rename the marca column to brand
# df2_with_brands.rename(columns={'marca': 'brand'}, inplace=True)
# export df to csv
df2_with_brands.to_csv('data.csv', index=False)


In [19]:
df2 = pd.read_csv('data.csv')

In [52]:
#show me column 15 and 16
df2.columns

Index(['Tipo Movimiento', 'Sucursal', 'Vendedor', 'Cliente', 'Rut Cliente',
       'Tipo de Documento', 'Numero Documento', 'Tracking Number',
       'Fecha Documento', 'Fecha y Hora Venta', 'Lista de Precio',
       'Tipo de Producto / Servicio', 'SKU', 'Producto / Servicio',
       'Otros Atributos', 'Precio Neto Unitario', 'Cantidad',
       'Subtotal Impuestos', 'Subtotal Neto', 'Subtotal Bruto',
       'Costo Neto Unitario', 'Margen', 'Costo Total Neto', '% Margen',
       'Detalle de Productos/Servicios Pack/Promo', 'Dirección Cliente',
       'Comuna Cliente', 'Ciudad Cliente', 'Email Cliente', 'marca'],
      dtype='object')

In [17]:
shopify_count = (df2['Sucursal'] == 'Uber').sum()
print(shopify_count)

1734


In [18]:
df2.loc[(df2['Vendedor'] == 'Loadingplay Connect'), 'Sucursal'] = 'Shopify'
shopify_count = (df2['Sucursal'] == 'Shopify').sum()
print(shopify_count)

75552


In [78]:
df2['Fecha Documento'] = df2['Fecha Documento'].str.replace('=', '')  # Remove "="
df2['Fecha Documento'] = df2['Fecha Documento'].str.replace('"', '')  # Remove extra quotes

# Step 2: Convert the cleaned strings to datetime objects
df2['Fecha Documento'] = pd.to_datetime(df2['Fecha Documento'], format='%d/%m/%Y')

In [50]:
import pandas as pd
df2 = pd.read_csv('data.csv')


In [51]:
unique_sucursal = df2['Sucursal'].unique()
unique_sucursal

array(['CASA MATRIZ', 'Shopify', 'TIENDA VITACURA',
       'EVENTOS - CASA MATRIZ', 'Uber'], dtype=object)

In [48]:
other_count = (df2['marca'] == 'Caffe Borbone').sum()
other_count
# Aca quede

124

In [46]:
import numpy as np

unique_values = np.sort(df2['marca'].unique())
unique_values


array(['A&F', 'ALBIERO', 'ALFA', 'ALPI', 'ANTONIO AMATO',
       'BELLA CONTADINA', 'BOD 12', 'BODEGA 12', 'BONI', 'BONOMELLI',
       'BORBNE', 'BORBONE', 'BORGHETTO', 'BOSCO D ORO', 'BUFALY', 'Boni',
       'Borbone', 'CANNAMELA', 'CAVICCHIOLI', 'CIEMME', 'CLEMENTE',
       'COLI', 'CORTE BUONA', 'CURTI', 'CURTIRISO', 'Caffe Borbone',
       'Corte Buona', 'DCORDI', 'DECORDI', 'DI MARTINO', 'DIDIESSE',
       'DIMARTINO', 'DOLCIARIA ACQUAVIVA', 'FALCONE', 'FERRARI',
       'FERRERO', 'FIORENTINI', 'GALLO', 'GASTOS VARIOS', 'GGI',
       'GI METAL', 'GI-METAL', 'GIMETAL', 'GOUR', 'GOURMITALIA',
       'GRANORO', 'GRAZIELLA', 'Gourmitalia', 'IMPRESOS JEMBA',
       'LA PICCOLA', 'LATTERIA SORRENTINA', 'LAZZARONI', 'LE 5 STAGIONI',
       'LOZZANO', 'LUX', 'Lazzaroni', 'MARINI', 'MONARI DEFERZONI',
       'MONARI DEFORZONI', 'MONARI FEDERZONI', 'MONTENEGRO', 'MORENO',
       'MOTTA', 'MULINO BIANCO', 'MURGELLA', 'MUTTI', 'Melegatti',
       'NEGRINI', 'NORTINDAL', 'NOVELLA', 'Negrini', 

In [42]:
df = df.iloc[5:]
df = df.rename(columns=df.iloc[0])
# Print df head


In [53]:
df.columns

AttributeError: 'list' object has no attribute 'columns'

In [47]:
# Show me the unique values of column 'A'
print(df['Sucursal'].unique())
# # Count the nan values in the column 'A'
# print(df['Sucursal'].isnull().sum())
# # Drop the rows with nan values in the column 'A'
# df = df.dropna(subset=['Sucursal'])
# print(df['Sucursal'].unique())


['CASA MATRIZ' 'EVENTOS - CASA MATRIZ' 'TIENDA VITACURA']


In [24]:
df['Fecha Documento'] = pd.to_datetime(df['Fecha Documento'], format='%d/%m/%Y')


In [27]:
#Sort the dataframe by column 'Fecha Documento' in ascending order
df = df.sort_values(by='Fecha Documento', ascending=True)
print(df.head(1))
#print value in the first row where Sucursal is 'CASA MATRIZ'
# print(df[df['Sucursal'] == 'CASA MATRIZ'].iloc[0])

    Tipo Movimiento         Sucursal       Vendedor      Cliente Rut Cliente  \
481           Venta  TIENDA VITACURA  MANUEL UGARTE  Sin cliente         NaN   

        Tipo de Documento  Numero Documento           Tracking Number  \
481  boleta electrónica t            104341  63ebea668bd0060549391861   

    Fecha Documento   Fecha y Hora Venta  ... Subtotal Bruto  \
481      2023-02-14  14/02/2023 05:09 PM  ...          11480   

    Costo Neto Unitario Margen Costo Total Neto % Margen  \
481                3552   6095             3552       63   

     Detalle de Productos/Servicios Pack/Promo  Dirección Cliente  \
481                                        NaN                NaN   

     Comuna Cliente  Ciudad Cliente  Email Cliente  
481             NaN             NaN            NaN  

[1 rows x 29 columns]


In [48]:
filtered_df = df[df['Sucursal'] == 'CASA MATRIZ']
total_branch = filtered_df['Subtotal Bruto'].sum()
print(total_branch)

1895580408


# BRANDS

In [40]:
# Supose df2 is the df ready just missing the columns brands

#df shape
df3 = pd.read_csv('productos.csv')
# Keep just Gourmitalia
df3 = df3[df3['proveedores_id'] == 1]
#  Rename the sku to match the other df
df3.rename(columns={'sku_code': 'SKU'}, inplace=True)
# Just keep the columns we need to merge with the main df SKU and marca
df3 = df3[['SKU', 'marca']]
#export to csv
df3.to_csv('brands.csv', index=False)
# Fix the int of SKU in the main df
# FIX DATE
df2['SKU'] = df2['SKU'].str.replace('=', '')  # Remove "="
df2['SKU'] = df2['SKU'].str.replace('"', '')  # Remove extra quotes
df2['SKU'] = df2['SKU'].astype(str)
# merge the two df
df2_with_marca = pd.merge(df2, df3[['SKU', 'marca']], on='SKU', how='left')


In [28]:
df3.columns

Index(['id', 'image_path', 'id_bsale', 'id_variante', 'proveedores_id',
       'sku_code', 'nombre_producto', 'descripcion', 'precio', 'categoria',
       'formato', 'stock', 'created_at', 'updated_at', 'cantidad', 'unidad',
       'estado', 'formato_id', 'marca', 'taxes'],
      dtype='object')

In [29]:
df2.columns

Index(['Tipo Movimiento', 'Sucursal', 'Vendedor', 'Cliente', 'Rut Cliente',
       'Tipo de Documento', 'Numero Documento', 'Tracking Number',
       'Fecha Documento', 'Fecha y Hora Venta', 'Lista de Precio',
       'Tipo de Producto / Servicio', 'SKU', 'Producto / Servicio',
       'Otros Atributos', 'Precio Neto Unitario', 'Cantidad',
       'Subtotal Impuestos', 'Subtotal Neto', 'Subtotal Bruto',
       'Costo Neto Unitario', 'Margen', 'Costo Total Neto', '% Margen',
       'Detalle de Productos/Servicios Pack/Promo', 'Dirección Cliente',
       'Comuna Cliente', 'Ciudad Cliente', 'Email Cliente'],
      dtype='object')

In [30]:
df3.shape


(1209, 20)

In [31]:
# Rename 'sku_code' in df3 to 'SKU'
df3.rename(columns={'sku_code': 'SKU'}, inplace=True)

/var/folders/8v/92m9nr2x2rlg853t_ypdtlrr0000gn/T/ipykernel_71591/2429081202.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3.rename(columns={'sku_code': 'SKU'}, inplace=True)


In [33]:
df2_with_marca = pd.merge(df2, df3[['SKU', 'marca']], on='SKU', how='left')


In [47]:
# Count the number of rows where 'Marca' is NaN
nan_count = df2_with_brands['marca'].isna().sum()

print(f"Number of rows with NaN in the 'Marca' column: {nan_count}")

# Show a row where marca is Nan
print(df2_with_brands[df2_with_brands['marca'].isna()].iloc[0])


Number of rows with NaN in the 'Marca' column: 7547
Tipo Movimiento                                                              Venta
Sucursal                                                               CASA MATRIZ
Vendedor                                                         REYNALDO CARDENAS
Cliente                                                    GRACIELA CANDIA KWASNIK
Rut Cliente                                                             14520286-8
Tipo de Documento                                            factura electrónica t
Numero Documento                                                            118817
Tracking Number                                                                NaN
Fecha Documento                                                2023-02-14 00:00:00
Fecha y Hora Venta                                          ="14/02/2023 07:47 AM"
Lista de Precio                                               ="MAYORISTA 11/2022"
Tipo de Producto / Servicio        

In [42]:
df2['SKU']

0         14024
1         14022
2         15203
3         14013
4         33005
          ...  
165646    65020
165647    33055
165648    65021
165649    33005
165650      nan
Name: SKU, Length: 165651, dtype: object